In [27]:
from IPython.display import display, Math
import pyvista
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
# import meshio
import dolfinx
from mpi4py import MPI
from dolfinx.io import gmshio
import numpy as np
# import gmsh
# from gmshModel.Model import RandomInclusionRVE as RVE
from dolfinx.io import XDMFFile,gmshio

In [28]:
# !pip install meshio
# ! pip install h5py
# ! pip install gmsh
# ! python3 -m pip install gmshModel
# ! pip list

In [29]:

# mesh_comm = MPI.COMM_WORLD
# model_rank = 0
# gmsh.initialize()

# facet_tags = {"left": 1, "right": 2, "top": 3, "crack": 4, "bottom_no_crack": 5}
# cell_tags = {"all": 20
# if mesh_comm.rank == model_rank:
#     model = gmsh.model()
#     model.add("Rectangle")
#     model.setCurrent("Rectangle")
#     # Create the points
#     p1 = model.geo.addPoint(0.0, 0.0, 0, lc)
#     p2 = model.geo.addPoint(Lcrack, 0.0, 0, lc)
#     p3 = model.geo.addPoint(Lx, 0, 0, lc)
#     p4 = model.geo.addPoint(Lx, Ly, 0, lc)
#     p5 = model.geo.addPoint(0, Ly, 0, lc)
#     # Create the lines
#     l1 = model.geo.addLine(p1, p2, tag=facet_tags["crack"])
#     l2 = model.geo.addLine(p2, p3, tag=facet_tags["bottom_no_crack"])
#     l3 = model.geo.addLine(p3, p4, tag=facet_tags["right"])
#     l4 = model.geo.addLine(p4, p5, tag=facet_tags["top"])
#     l5 = model.geo.addLine(p5, p1, tag=facet_tags["left"])
#     # Create the surface
#     cloop1 = model.geo.addCurveLoop([l1, l2, l3, l4, l5])
#     surface_1 = model.geo.addPlaneSurface([cloop1])

#     # Define the mesh size and fields for the mesh refinement
#     model.mesh.field.add("Distance", 1)
#     model.mesh.field.setNumbers(1, "NodesList", [p2])
#     # SizeMax -                   / ------------------
#     #                            /
#     # SizeMin -o----------------/
#     #          |                |  |
#     #        Point        DistMin   DistMax
#     model.mesh.field.add("Threshold", 2)
#     model.mesh.field.setNumber(2, "IField", 1)
#     model.mesh.field.setNumber(2, "LcMin", lc / refinement_ratio)
#     model.mesh.field.setNumber(2, "LcMax", lc)
#     model.mesh.field.setNumber(2, "DistMin", dist_min)
#     model.mesh.field.setNumber(2, "DistMax", dist_max)
#     model.mesh.field.setAsBackgroundMesh(2)
#     gmsh.option.setNumber("General.Verbosity", 3)
#     model.geo.synchronize()

#     # Assign mesh and facet tags
#     surface_entities = [entity[1] for entity in model.getEntities(2)]
#     model.addPhysicalGroup(2, surface_entities, tag=cell_tags["all"])
#     model.setPhysicalName(2, 2, "Rectangle surface")
#     model.mesh.generate(gdim)

#     for key, value in facet_tags.items():
#         model.addPhysicalGroup(1, [value], tag=value)
#         model.setPhysicalName(1, value, key)

In [30]:
# msh, cell_tags, facet_tags = gmshio.model_to_mesh(
#     model, mesh_comm, model_rank, gdim=gdim
# )
# msh.name = "rectangle"
# cell_tags.name = f"{msh.name}_cells"
# facet_tags.name = f"{msh.name}_facets"

In [31]:
# pyvista.set_jupyter_backend("HTML")
# pyvista.start_xvfb()


# # Extract topology from mesh and create pyvista mesh
# topology, cell_types, x = dolfinx.plot.vtk_mesh(msh)
# grid = pyvista.UnstructuredGrid(topology, cell_types, x)

# plotter = pyvista.Plotter()
# plotter.add_mesh(grid, show_edges=True)
# plotter.show()

In [32]:
# def create_mesh(mesh, cell_type, prune_z=False):
#     cells = mesh.get_cells_type(cell_type)
#     cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
#     points = mesh.points[:, :2] if prune_z else mesh.points
#     out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read": [cell_data.astype(np.int32)]})
#     return out_mesh

In [33]:

# # Convert Gmsh mesh to XDMF format
# msh = meshio.read("mesh.msh")
# # meshio.write("mesh.xdmf", msh)

# # Load mesh in FEniCS

# # gmshio.extract_topology_and_markers(msh)

# # mesh, cell_tags, facet_tags = gmshio.read_from_msh("mesh.msh", MPI.COMM_WORLD,0, gdim=2)

# # # Visualize the mesh
# import pyvista as pv
# from dolfinx.plot import vtk_mesh

# mesh = meshio.read('mesh.msh')
# mymesh = create_mesh(mesh, "quadrilateral")
# topology, cell_types, x = vtk_mesh(mymesh,mymesh.topology.dim)

# grid = pv.UnstructuredGrid(topology, cell_types, x)

# # grid.plot(show_edges=True)
# pv.start_xvfb()
# pv.set_jupyter_backend('html')

# plottter = pv.Plotter()

# plottter.add_mesh(grid, show_edges=True)
# plottter.view_xy()
# plottter.show()

In [43]:
# Loading of the RVE class
# Before the model and mesh generation can start, the required class has to be
# loaded. In this case it is the class RandomInclusionRVE
from gmshModel.Model import RandomInclusionRVE 


# Initialization of the RVE
# In order to generate a mesh for RVEs with randomly placed inclusions, relevant
# data have to be passed for the initialization of a new object instance. For
# RVEs of the type under consideration, the following parameters are possible:
#
# size: list/array (mandatory)
#   array defining the size of the RVE in the individual directions
#   -> size=[L_x, L_y, L_z]
#
# inclusionSets: list/array (mandatory)
#   array defining the relevant information (radius and amount) for the individual
#   groups of spherical inclusions to be placed
#   -> inclusionSets=[[r_1, n_1] [r_2, n_2], ..., [r_n, n_n]]
#
# inclusionType: string (mandatory)
#   string defining the type of inclusions within the RVE
#
# origin: list/array (optional)
#   array defining the origin of the RVE
#   -> origin=[O_x, O_y, O_z]
#
# periodicityFlags: list/array (optional)
#   array with flags (0/1) whether the current coordinate direction has to be
#   treated as periodic
#   periodicityFlags=[0/1, 0/1, 0/1]
#
# domainGroup: string (optional)
#   string defining which group the geometric objects defining the domain belong
#   to (to reference this group within boolean operations)
#
# inclusionGroup: string (optional)
#   string defining which group the geometric objects defining the inclusions
#   belong to (to reference this group within boolean operations)
#
# gmshConfigChanges: dict (optional)
#   dictionary for user updates of the default Gmsh configuration
#
initParameters={                                                                # save all possible parameters in one dict to facilitate the method call
    "inclusionSets": [0.15, 5],                                       # place 8 inclusions with radius 1 and 10 inclusions with radius 0.5
    "inclusionType": "Circle",                                                  # define inclusionType as "Circle"
    "size": [1, 1, 0],                                                       # set RVE size to [10,10,10]
    "origin": [0, 0, 0],                                                        # set RVE origin to [0,0,0]
    "periodicityFlags": [1, 1, 1],                                              # define all axis directions as periodic
    "domainGroup": "domain",                                                    # use "domain" as name for the domainGroup
    "inclusionGroup": "inclusions",                                             # use "inclusions" as name for the inclusionGroup
    "gmshConfigChanges": {"General.Terminal": 0,                                # deactivate console output by default (only activated for mesh generation)
                          "Mesh.CharacteristicLengthExtendFromBoundary": 0,     # do not calculate mesh sizes from the boundary by default (since mesh sizes are specified by fields)
    }
}
testRVE=RandomInclusionRVE(**initParameters)


# Gmsh model generation
# After all parameters for the RVE are set, the Gmsh model can be generated.
# This process involves the definition of geometric objects, their combination
# to more complex shapes using boolean operations and the definition of physical
# groups, i.e. groups of elements that belong to the same material or part of
# the boundary. For RVEs with randomly placed inclusions, only the placement
# options can be changed by the user. To this end, the possible parameters are:
#
# placementOptions: dict (optional)
#   user updates for the inclusion placement algorithm
#
modelingParameters={                                                            # save all possible parameters in one dict to facilitate the method call
    "placementOptions": {"maxAttempts": 10000,                                  # maximum number of attempts to place one inclusion
                         "minRelDistBnd": 0.2,                                  # minimum relative (to inclusion radius) distance to the domain boundaries
                         "minRelDistInc": 0.4,                                  # minimum relative (to inclusion radius) distance to other inclusions}
    }
}
testRVE.createGmshModel(**modelingParameters)


# Gmsh mesh creation
# After the model has been created using the Gmsh-Python-API, the meshing
# can be performed. To this end, refinement fields defining the mesh sizes
# within the model have to be calculated and added to the Gmsh model. Once, the
# mesh sizes are specified,the mesh can be generated. Available parameters are:
#
# threads: int
#   number of threads to use for the meshing procedure
# refinementOptions: dict (optional)
#   dictionary containing user updates for the refinement field calculation
#
meshingParameters={                                                             # save all possible parameters in one dict to facilitate the method call
    "threads": None,                                                            # do not activate parallel meshing by default
    "refinementOptions": {"maxMeshSize": "auto",                                # automatically calculate maximum mesh size with built-in method
                          "inclusionRefinement": True,                          # flag to indicate active refinement of inclusions
                          "interInclusionRefinement": True,                     # flag to indicate active refinement of space between inclusions (inter-inclusion refinement)
                          "elementsPerCircumference": 50,                       # use 18 elements per inclusion circumference for inclusion refinement
                          "elementsBetweenInclusions": 3,                       # ensure 3 elements between close inclusions for inter-inclusion refinement
                          "inclusionRefinementWidth": 10,                        # use a relative (to inclusion radius) refinement width of 3 for inclusion refinement
                          "transitionElements": "auto",                         # automatically calculate number of transitioning elements (elements in which tanh function jumps from h_min to h_max) for inter-inclusion refinement
                          "aspectRatio": 1.5                                    # aspect ratio for inter-inclusion refinement: ratio of refinement in inclusion distance and perpendicular directions
    }
}
testRVE.createMesh(**meshingParameters)


# Save resulting mesh to file
# The mesh is generated and can be saved to a file. To this end, only the file
# name - possibly containing a directory and the extension of the wanted mesh
# format - has to be passed. The package supports all mesh file formats that are
# supported by meshio. If no filename is passed, meshes are stored to the current
# directory using the unique model name and the default mesh file format (.msh)
#
testRVE.saveMesh("randomInclusions2DCirlce.msh")


# Show resulting mesh
# To check the generated mesh, the result can also be visualized using built-in
# methods.
#
testRVE.visualizeMesh()


# Close Gmsh model
# For a proper closing of the Gmsh-Python-API, thAPI has to be finalized. This
# can be achieved by calling the close() method of the model
#
testRVE.close()

Info    : Reconstructing periodicity for curve connection 12 - 8
Info    : Reconstructing periodicity for curve connection 15 - 5
Info    : Reconstructing periodicity for curve connection 17 - 10
Info    : Reconstructing periodicity for curve connection 21 - 16
Info    : Reconstructing periodicity for curve connection 22 - 11
Info    : Reconstructing periodicity for curve connection 27 - 31
Info    : Reconstructing periodicity for curve connection 28 - 30
Info    : Reconstructing periodicity for curve connection 29 - 26
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Ellipse)
Info    : [ 10%] Meshing curve 7 (Ellipse)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Ellipse)
Info    : [ 20%] Meshing curve 10 (Line)
Info    : [ 30%] Meshing curve 11 (Line)
Info    : [ 30%] Meshing curve 12 (Line) as a copy of curve 8
Info    : [ 30%] Meshing curve 13 (Ellipse)
Info    : [ 40%] Meshing curve 14 (Ellipse)
Info    :

MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


Widget(value='<iframe src="http://localhost:36755/index.html?ui=P_0x7f27e01ca490_7&reconnect=auto" class="pyvi…

In [44]:
# with XDMFFile(MPI.COMM_WORLD, "randomInclusions2DCirlce.xdmf", "r") as xdmf:
#     mesh = xdmf.read_mesh(name="Grid")
#     ct = xdmf.read_meshtags(mesh, name="Grid")
# mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)
mesh, cell_markers, facet_markers = gmshio.read_from_msh("randomInclusions2DCirlce.msh", MPI.COMM_WORLD, gdim=2)


unique_markers = np.unique(cell_markers.values)


Info    : Reading 'randomInclusions2DCirlce.msh'...
Info    : 58 entities
Info    : 3094 nodes
Info    : 6186 elements
Info    : Done reading 'randomInclusions2DCirlce.msh'


In [36]:
# pyvista.set_jupyter_backend("html")
# pyvista.start_xvfb()


# # Extract topology from mesh and create pyvista mesh
# topology, cell_types, x = dolfinx.plot.vtk_mesh(mesh)
# grid = pyvista.UnstructuredGrid(topology, cell_types, x)

# plotter = pyvista.Plotter()
# plotter.add_mesh(grid, show_edges=True)
# plotter.show()




In [37]:
# ! pip install pyvista trame ipywidgets vtk
# ! pip install --upgrade ipykernel
# ! pip install trame-vtk trame-jupyter


In [38]:
mesh.topology.connectivity(2,1)
mesh.geometry.x.shape,mesh.geometry.dofmap.shape

((865, 3), (1624, 3))

In [39]:
mesh.topology.create_connectivity(2, 0)
conn = mesh.topology.connectivity(2, 0)
conn

<AdjacencyList> with 1624 nodes
  0: [0 1 2 ]
  1: [1 3 2 ]
  2: [4 3 1 ]
  3: [3 5 2 ]
  4: [6 3 4 ]
  5: [2 5 7 ]
  6: [6 5 3 ]
  7: [4 8 6 ]
  8: [7 5 9 ]
  9: [10 5 6 ]
  10: [8 11 6 ]
  11: [7 9 12 ]
  12: [9 5 10 ]
  13: [13 10 6 ]
  14: [14 11 8 ]
  15: [6 11 13 ]
  16: [12 9 15 ]
  17: [10 16 9 ]
  18: [13 16 10 ]
  19: [17 11 14 ]
  20: [13 11 17 ]
  21: [18 19 20 ]
  22: [20 21 14 ]
  23: [12 15 22 ]
  24: [9 16 15 ]
  25: [23 16 13 ]
  26: [21 17 14 ]
  27: [13 17 24 ]
  28: [20 19 21 ]
  29: [25 19 18 ]
  30: [15 26 22 ]
  31: [15 16 27 ]
  32: [24 23 13 ]
  33: [27 16 23 ]
  34: [24 17 21 ]
  35: [21 19 28 ]
  36: [18 29 25 ]
  37: [28 19 25 ]
  38: [30 29 18 ]
  39: [22 26 31 ]
  40: [15 27 26 ]
  41: [32 23 24 ]
  42: [23 33 27 ]
  43: [24 21 34 ]
  44: [21 28 34 ]
  45: [25 29 35 ]
  46: [36 28 25 ]
  47: [37 29 30 ]
  48: [31 26 38 ]
  49: [27 39 26 ]
  50: [34 32 24 ]
  51: [32 33 23 ]
  52: [27 33 40 ]
  53: [34 28 36 ]
  54: [36 25 35 ]
  55: [35 29 37 ]
  56: [41 3